In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-0.5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="cuda"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

#model.config.output_attentions = True  # 返回注意力权重
model.config.output_hidden_states = True  # 返回隐藏状态（可选）


Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [2]:
prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512,
    output_hidden_states=True
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

/home/maosen/miniconda3/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:817: UserWarning: `return_dict_in_generate` is NOT set to `True`, but `output_hidden_states` is. When `return_dict_in_generate` is not `True`, `output_hidden_states` is ignored.
  warnings.warn(


In [9]:
text = "Hello, world!"
inputs = tokenizer(text, return_tensors='pt')
inputs.to('cuda')
outputs = model(**inputs, output_hidden_states=True)

In [6]:
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512,
    output_hidden_states=True,
    return_dict_in_generate=True
)

In [11]:
generated_ids['hidden_states'][0]

(tensor([[[ 0.0014, -0.0084,  0.0073,  ..., -0.0032, -0.0128,  0.0187],
          [ 0.0293,  0.0210,  0.0073,  ..., -0.0014, -0.0273, -0.0216],
          [-0.0077,  0.0312, -0.0193,  ...,  0.0177,  0.0029, -0.0125],
          ...,
          [ 0.0014, -0.0084,  0.0073,  ..., -0.0032, -0.0128,  0.0187],
          [ 0.0037,  0.0215,  0.0232,  ..., -0.0165,  0.0001, -0.0109],
          [-0.0077,  0.0312, -0.0193,  ...,  0.0177,  0.0029, -0.0125]]],
        device='cuda:0', dtype=torch.bfloat16),
 tensor([[[ 0.5156,  0.0649, -0.3984,  ...,  0.1099,  0.4160,  0.2891],
          [-0.7344, -0.0742, -0.1592,  ...,  0.0918, -0.3809, -0.0508],
          [-0.1055,  0.0820,  0.0200,  ..., -0.1123, -0.0459, -0.0469],
          ...,
          [ 0.3301,  0.1309, -0.5898,  ..., -0.1631,  0.5508,  0.1318],
          [ 0.1611, -0.1299,  0.2578,  ..., -0.1226, -0.0557, -0.0762],
          [-0.1211,  0.0928, -0.0152,  ..., -0.0786,  0.0420, -0.0430]]],
        device='cuda:0', dtype=torch.bfloat16),
 tenso